In [1]:
import torch
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor
import os
import numpy as np

# prepare train/test data
create train and test from original.csv

In [2]:
test_df = pd.read_csv('original.csv')
df = test_df
len(test_df), len(df)

(99, 99)

In [4]:
mask = []
for i in range(len(np.array(df['input_text']))):
    if np.array(df['input_text'])[i] in np.array(test_df['input_text']):
        mask.append(False)
    else:
        mask.append(True)
mask = np.array(mask)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False])

In [11]:
df = test_df
mask = np.random.rand(len(df)) < 0.8
train_df = df[mask]
test_df = df[~mask]
train_df.to_csv('train_df.csv')
test_df.to_csv('test_df.csv')

## figure out if there are long inputs

In [6]:
ars =  np.array(df['input_text'])
mas = []
for a in ars:
    mas.append(len(a))
mas = np.array(mas)
len(np.where(mas>400)[0])

0

## instantiate model and tokenizer

In [5]:
import torch
if torch.cuda.is_available():
    dev = torch.device("cuda:2")
    print("Running on the GPU")
else:
    dev = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [6]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)
model.to(dev)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [9]:
# model.load_state_dict(torch.load('pytorch_model_categories.bin'))
#model.load_state_dict(torch.load('pytorch_model_categories_87_precision.bin'))

In [7]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))


In [8]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

## create test batches from test_df

In [12]:
batch_size = 16
num_of_batches=len(test_df)/batch_size
num_of_batches = int(num_of_batches)
test_inp_batches = []
test_label_batches =[]
for i in range(num_of_batches):
    
    new_df=test_df[i*batch_size:i*batch_size+batch_size]
    inputbatch=[]
    labelbatch=[]
    for indx,row in new_df.iterrows():
          input = row[training_column]
          labels = row['target_text'] 
          inputbatch.append(input)
          labelbatch.append(labels)
    test_inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
    test_labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
    test_inputbatch=test_inputbatch.to(dev)
    test_labelbatch=test_labelbatch.to(dev)
    test_inp_batches.append(test_inputbatch)
    test_label_batches.append(test_labelbatch)

## define loss metrics

In [13]:
import math
def generate(texts, model, tokenizer, targets):
    
    gc.collect()
    torch.cuda.empty_cache()
    batch_size=256
    num_of_batches=len(texts)*1.0/batch_size
    num_of_batches = math.ceil(num_of_batches)
    
    outputs = []
    targs = []
    for i in range(num_of_batches):
        #print(i, len(outputs), len(targs))
        inputbatch = []
        for text in texts[i*batch_size:i*batch_size+batch_size]:
            input = text  
            inputbatch.append(input)
        try:
            inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
            inputbatch=inputbatch.to(dev)
            model.to(dev)
            model.eval()
            outs = model.generate(inputbatch)
            outputs.extend(outs)
            targs.extend(targets[i*batch_size:i*batch_size+batch_size])
        except Exception as e:
            gc.collect()
            torch.cuda.empty_cache()
            continue
    return [tokenizer.decode(out) for out in outputs], targs

def evaluate(inps, targetss, model, tokenizer):
    precisions = []
    recalls = []
    preds, targets = generate(inps, model, tokenizer, targetss)
    for i in range(len(targets)):
        target = targets[i]
        pred = preds[i]#generate(inp, model, tokenizer)
        p, r = rouge_n(target, pred, 1)
        precisions.append(p)
        recalls.append(r)
    return np.mean(precisions), np.mean(recalls)

def rouge_n(target, pred, n):
    target_list = target.split(' ')
    pred_list = pred.split(' ')
    n_gram_target = get_n_gram_list(n, target)
    n_gram_pred = get_n_gram_list(n, pred)
    match = 0
    
    for i in range(min(len(n_gram_target),len(n_gram_pred))):
        if n_gram_target[i] in n_gram_pred:
            match += 1
    precision = match*1.0/len(n_gram_pred)
    recall = match*1.0/len(n_gram_target)
    return [precision, recall]
import re
def get_n_gram_list(n, text):
    text = text.replace('/pad>', '')
    text = text.replace('</s>', '').strip()
    target_list = text.split(' ')
    n_gram_target = []
    for i in range(len(target_list)-n+1):
        lis = [ re.sub(r'[^\w\s]', '', w.strip().lower()) for w in target_list[i:i+n]]
        n_gram_target.append(' '.join(lis))
    return n_gram_target
import nltk, string
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')
stop_words = list(set(stopwords.words("english")))
stop_words += list(string.punctuation)
stop_words += ['__', '___']
def getlsa(texts):
    tokenizer = RegexpTokenizer(r'\b\w{3,}\b')
    tfidf = TfidfVectorizer(lowercase=True, 
                            stop_words=stop_words, 
                            tokenizer=tokenizer.tokenize, 
#                             max_df=0.2,
#                             min_df=0.02
                           )
    tfidf_train_sparse = tfidf.fit_transform(texts)
    tfidf_train_df = pd.DataFrame(tfidf_train_sparse.toarray(), 
                            columns=tfidf.get_feature_names())
    lsa_obj = TruncatedSVD(n_components=10, n_iter=1000, random_state=42)
    tfidf_lsa_data = lsa_obj.fit_transform(tfidf_train_df)
    return tfidf_lsa_data
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
def sim(v1, v2):
    return np.dot(v1,v2)
def lsa_evaluate(inps, targetss, model, tokenizer, concepts):
    preds, targets = generate(inps, model, tokenizer, targetss)
    real_score = 0
    for pred in preds:
        score = 0
        for concept in concepts:
            score += sim(concept, pred)
        real_score += score*1.0/len(concepts)
    
    return real_score
        

[nltk_data] Downloading package stopwords to /Users/vin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## define model

In [9]:
optimizer = Adafactor(model.parameters(),lr=1e-4,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

## read dataset

In [16]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')
training_column = "cat_conc_sec"

## training loop

In [17]:
#Check examples after every epoch
import gc
import traceback
num_of_epochs = 1
batch_size=16
num_of_batches=len(train_df)/batch_size
num_of_batches = int(num_of_batches)

#Sets the module in training mode
model.train()

loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
    print('{precision, recall} = ', evaluate(np.array(test_df[training_column]), np.array(test_df['target_text']), model, tokenizer))
    running_loss=0

    out = display(progress(1, num_of_batches+1), display_id=True)
    
    for i in range(num_of_batches):
        if i%500 == 0:
            print(i, num_of_batches)
        new_df=train_df[i*batch_size:i*batch_size+batch_size]
        inputbatch=[]
        labelbatch=[]
        for indx,row in new_df.iterrows():
            input = row[training_column]
            labels = row['target_text']
            inputbatch.append(input)
            labelbatch.append(labels)
        inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
        labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
        inputbatch=inputbatch.to(dev)
        labelbatch=labelbatch.to(dev)

        # clear out the gradients of all Variables 
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        # Forward propogation
        try:
            outputs = model(input_ids=inputbatch, labels=labelbatch)

            loss = outputs.loss
            loss_num=loss.item()
            logits = outputs.logits
            running_loss+=loss_num
            if i%10 ==0:      
                loss_per_10_steps.append(loss_num)
            out.update(progress(loss_num,i, num_of_batches+1))

            # calculating the gradients
            loss.backward()

            #updating the params
            optimizer.step()
            
            gc.collect()
            torch.cuda.empty_cache()
        except Exception as e:
            print(str(e))
            #traceback.print_exc()
            torch.save(model.state_dict(),'pytorch_model_categories.bin')
            gc.collect()
            torch.cuda.empty_cache()
            continue

    running_loss=running_loss/int(num_of_batches)
    torch.save(model.state_dict(),'pytorch_model_categories.bin')
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))
    ix = 0
    test_output_loss = 0
    while ix < len(test_inp_batches):
        if ix%200==0:
            print(ix, len(test_inp_batches))
        gc.collect()
        torch.cuda.empty_cache()
        try:
            test_output_loss += model(input_ids=test_inp_batches[ix], labels=test_label_batches[ix]).loss.item()
            ix += 1
        except Exception as e:
            print(str(e))
            gc.collect()
            torch.cuda.empty_cache()
            ix += 1
        
    print('Validation Loss:', test_output_loss/int(num_of_batches))
  

Running epoch: 1
{precision, recall} =  (0.03472222222222222, 0.057638888888888885)


0 5


/Users/vin/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:557: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1005.)
  exp_avg_sq_row.mul_(beta2t).add_(1.0 - beta2t, update.mean(dim=-1))


KeyboardInterrupt: 

# break

In [ ]:
#precision: when we predict the word and it's present in target set
#Recall: When word is in target set and we predict it as well
test_df.to_csv('test_df.csv')

In [ ]:
torch.save(model.state_dict(),'42_recall_51_lsa.bin')

In [ ]:
print('{precision, recall} = ', evaluate(np.array(train_df[training_column]), np.array(train_df['target_text']), model, tokenizer))

In [ ]:
#test_df = df
print('{precision, recall} = ', evaluate(np.array(test_df[training_column]), np.array(test_df['target_text']), model, tokenizer)), #0.0468

In [ ]:
Feed like this ? Incorporate medical concepts in the following way ?
term 1 | term 2 |
term 1 <concept1> | term 2 <concept2>

instead of using terms, just use concepts ?

train the model specifically for each category (e.g.: symptom, medications, family history)

In [ ]:
print(np.array(test_df['input_text'])[ix])

In [ ]:
test_df[15:17]

In [ ]:
model.to(dev)
ix = 500
print(np.array(test_df['target_text'])[ix])
print(np.array(test_df[training_column])[ix])
generate(np.array(test_df[training_column])[ix:ix+1], model, tokenizer, np.array(test_df['target_text'])[ix:ix+1])

In [ ]:
# Rouge modified evaluation

In [ ]:
preds, targets = generate(np.array(test_df['input_text']), model, tokenizer, np.array(test_df['target_text']))
pred_lsas = getlsa(preds)
target_lsas = getlsa(targets)

In [ ]:
pred_lsas.shape, len(preds), target_lsas.shape, len(targets)

In [ ]:
k = 9
print(np.array(test_df['input_text'])[k], np.array(test_df['target_text'])[k], preds[k])
sim(pred_lsas[k], target_lsas[k])

In [ ]:
su = 0
for k in range(len(preds)):
    su += sim(pred_lsas[k], target_lsas[k])
su, su/len(preds)

In [ ]:
precisions = []
recalls = []
preds, targets = generate(np.array(test_df['input_text']), model, tokenizer, np.array(test_df['target_text']))
print('Predictions done')

In [ ]:
for i in range(len(targets)):
    target = targets[i]
    pred = preds[i]#generate(inp, model, tokenizer)
    p, r = rouge_n(target, pred, 1)
    precisions.append(p)
    recalls.append(r)
print(np.mean(precisions), np.mean(recalls))

In [ ]:
np.mean(precisions), np.mean(recalls)

In [ ]:
type(model)

## Akilesh's Playground

In [ ]:
model.to(dev)
ix = 500
print(np.array(test_df['target_text'])[ix])
print(np.array(test_df[training_column])[ix])
generate(np.array(test_df[training_column])[ix:ix+1], model, tokenizer, np.array(test_df['target_text'])[ix:ix+1])

In [ ]:
train_df.head()

In [ ]:
import time

inp = np.array(test_df[training_column])[ix:ix+1]
targ = np.array(test_df['target_text'])[ix:ix+1]
# ai, target = generate(, model, tokenizer, )


In [ ]:
df = train_df.append(test_df)
for i in range(len(df['cat_conc_sec'])):
    if 'Chief' in np.array(df['cat_conc_sec'])[i]:
        print(np.array(df['cat_conc_sec'])[i])

In [ ]:
import time
start = time.time()
inp = ['Subjective:Action:Medications:go on | Subjective:Name:Medications:robitussin']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print(ai)
print(time.time() - start, " seconds to generate this sentence.")

In [ ]:
import time
start = time.time()
inp = ['Plan:Timeframe:FollowUp:next week']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print(ai)
print(time.time() - start, " seconds to generate this sentence.")

In [ ]:
import time
start = time.time()
inp = ['Subjective:Denies:Symptom:allergies']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print(ai)
print(time.time() - start, " seconds to generate this sentence.")

In [ ]:
import time
start = time.time()
inp = ['Subjective:Complaint:ChiefComplaint:obesity']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print(ai)
print(time.time() - start, " seconds to generate this sentence.")

In [ ]:
import time
start = time.time()
inp = ['Statistic Name: Blood Oxygen | Statistic Value: high']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print(ai)
print(time.time() - start, " seconds to generate this sentence.")

In [ ]:
import time
start = time.time()
phaseIIoutput = [('go', 'Plan_MedicationsAction'), ('robitussin', 'Plan_MedicationsName'), ('adding', 'Plan_MedicationsAction'), ('go on', 'Plan_MedicationsAction')]

inp = ['Action: go | Name: robitussin | Action: Adding | Action: go on']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print(ai)
print(time.time() - start, " seconds to generate this sentence.")

In [ ]:
target

In [ ]:
start = time.time()
phaseIIoutput = [('go', 'Plan_MedicationsAction'), ('robitussin', 'Plan_MedicationsName'), ('adding', 'Plan_MedicationsAction'), ('go on', 'Plan_MedicationsAction')]

inp = ['Location: lungs']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print(ai)
print(time.time() - start, " seconds to generate this sentence.")

In [ ]:
start = time.time()

phaseIIouptut = [('constantly', 'Subjective_SymptomFrequency'), ('coughing', 'Subjective_SymptomName'), ('every day', 'Subjective_SymptomFrequency')]

inp = ['Name: constantly | Name: coughing | Frequency: every day']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print(ai)
print(time.time() - start, " seconds to generate this sentence.")

In [ ]:
start = time.time()

phaseIIouptut = [('constantly', 'Subjective_SymptomFrequency'), ('coughing', 'Subjective_SymptomName'), ('every day', 'Subjective_SymptomFrequency')]

inp = ['Denies: allergies']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print(ai)
print(time.time() - start, " seconds to generate this sentence.")

In [ ]:
start = time.time()

phaseIIouptut = [('constantly', 'Subjective_SymptomFrequency'), ('coughing', 'Subjective_SymptomName'), ('every day', 'Subjective_SymptomFrequency')]

inp = ['Denies: distress']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print("AI told to write about: ", inp[0])
print("AI Generated Sentence: ", ai[0].replace("<pad>", "").replace("</s>", ""))
print(time.time() - start, " seconds to generate this sentence.")

In [ ]:
start = time.time()

phaseIIouptut = [('constantly', 'Subjective_SymptomFrequency'), ('coughing', 'Subjective_SymptomName'), ('every day', 'Subjective_SymptomFrequency')]

inp = ['Action: on']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print("AI told to write about: ", inp[0])
print("AI Generated Sentence: ", ai[0].replace("<pad>", "").replace("</s>", ""))
print(time.time() - start, " seconds to generate this sentence.")

In [ ]:
start = time.time()

phaseIIouptut = [('constantly', 'Subjective_SymptomFrequency'), ('coughing', 'Subjective_SymptomName'), ('every day', 'Subjective_SymptomFrequency')]

inp = ['Type: xray | Location: right knee | Findings: fractured tibia']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print("AI told to write about: ", inp[0])
print("AI Generated Sentence: ", ai[0].replace("<pad>", "").replace("</s>", ""))
print(time.time() - start, " seconds to generate this sentence.")

In [ ]:
start = time.time()

phaseIIouptut = [('constantly', 'Subjective_SymptomFrequency'), ('coughing', 'Subjective_SymptomName'), ('every day', 'Subjective_SymptomFrequency')]

inp = ['Date: 1/12/1944 | Action: follow up']
targ = ['Their temperature is 92.']
ai, target = generate(np.array(inp), model, tokenizer, np.array(targ))
print("AI told to write about: ", inp[0])
print("AI Generated Sentence: ", ai[0].replace("<pad>", "").replace("</s>", ""))
print(time.time() - start, " seconds to generate this sentence.")